# Returns a Malaria prevalence raster using a linear function

In [2]:
import rioxarray
import numpy as np

In [12]:
# Covariates
lst = rioxarray.open_rasterio("Uganda Standardized Rasters/standard_uganda_LSTday_2km_2018.tif")
rainfall = rioxarray.open_rasterio("Uganda Standardized Rasters/standard_uganda_Rainfall_CHIRPS_2km_2018.tif")
elevation = rioxarray.open_rasterio("Uganda Standardized Rasters/standard_uganda_elevation_2km.tif")

In [9]:
lst = lst.sel(band=1)

In [4]:
# Reprojects covariates
elevation = elevation.rio.reproject_match(lst)

In [5]:
elevation = elevation.assign_coords({
    "x": lst.x,
    "y": lst.y
})

In [176]:
# Arbitrary function
malaria_logit_prevalence = (-1.5 + .5 * lst + .75 * rainfall - .25 * elevation)

In [177]:
malaria_prevalence_probability = 1 / (1 + np.exp(-malaria_logit_prevalence))

In [178]:
malaria_prevalence_probability.rio.to_raster("Uganda Malaria Data/uganda_mock_malaria_prevelance_2km_2018.tif")